In [1]:
import pandas as pd
import requests

from datetime import datetime
import time

from scrapy import Selector

In [2]:
url = 'https://en.wikipedia.org/wiki/Nasdaq-100'

html = requests.get(url).text
sel = Selector(text = html)

nasdaq100_df = pd.DataFrame()

for n in range(2,104):
    ticker = sel.xpath('//table[@id = "constituents"]/tbody/tr[{}]/td[2]'.format(n)).extract()[0].strip('</td>')
    company_name = sel.xpath('//table[@id = "constituents"]/tbody/tr[{}]/td[1]/a/@title'.format(n)).get()
    industry = sel.xpath('//table[@id = "constituents"]/tbody/tr[{}]/td[3]'.format(n)).get().strip('</td>')
    sub_industry = sel.xpath('//table[@id = "constituents"]/tbody/tr[{}]/td[3]'.format(n)).get().strip('</td>')
    new_list = [ticker, company_name, industry, sub_industry]
    pd_series_row = pd.Series(new_list, index = ['ticker', 'company_name', 'industry', 'sub_industry'])
    nasdaq100_df = nasdaq100_df.append(pd_series_row, ignore_index=True)
    print(ticker, end = ' ')

nasdaq100_df.head(4)

ATVI ADBE AMD ALGN GOOGL GOOG AMZN AEP AMGN ADI ANSS AAPL AMAT ASML TEAM ADSK ADP BIDU BIIB BKNG AVGO CDNS CDW CERN CHTR CHKP CTAS CSCO CTSH CMCSA CPRT COST CRWD CSX DXCM DOCU DLTR EBAY EA EXC FAST FISV FOXA FOX GILD HON IDXX ILMN INCY INTC INTU ISRG JD KDP KLAC KHC LRCX LULU MAR MRVL MTCH MELI FB MCHP MU MSFT MRNA MDLZ MNST NTES NFLX NVDA NXPI ORLY OKTA PCAR PAYX PYPL PTON PEP PDD QCOM REGN ROST SGEN SIRI SWKS SPLK SBUX SNPS TMUS TSLA TXN TCOM VRSN VRSK VRTX WBA WDAY XEL XLNX ZM 

,company_name,industry,sub_industry,ticker
0,Activision Blizzard,Communication Services,Communication Services,ATVI
1,Adobe Inc.,Information Technology,Information Technology,ADBE
2,Advanced Micro Devices,Information Technology,Information Technology,AMD
3,Align Technology,Health Care,Health Care,ALGN


In [3]:
def unix_to_date(unix_timestamp):
    ts = int(unix_timestamp)
    return datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d')

In [4]:
with open('/Users/ilya/Desktop/keys/yahoo_finance.txt','r') as file:
    key = file.readlines()

KEY = key[0]

In [5]:
def get_weekly_history_df(symbol):
    get_history_url = "https://yh-finance.p.rapidapi.com/stock/v3/get-historical-data"

    querystring = {"symbol":symbol,"region":"US"}

    headers = {
    'x-rapidapi-host': "yh-finance.p.rapidapi.com",
    'x-rapidapi-key': KEY
    }

    response = requests.request("GET", get_history_url, headers=headers, params=querystring)

    json_response = response.json()

    weekly_history_df = pd.DataFrame(json_response['prices'])[['date','close']]
    weekly_history_df.loc[:,'date'] = weekly_history_df['date'].apply(lambda x: unix_to_date(x))

    weekly_history_df.loc[:,'week'] = [pd.Timestamp(date).week for date in weekly_history_df['date']]
    weekly_history_df.loc[:,'year'] = [pd.Timestamp(date).year for date in weekly_history_df['date']]

    weekly_history_df.loc[:,'close'] = weekly_history_df['close'].round(1)

    return weekly_history_df[['date','year','week','close']]

In [60]:
balance_sheet_url = "https://yh-finance.p.rapidapi.com/stock/v2/get-balance-sheet"

querystring = {"symbol":"ATVI","region":"US"}

headers = {
    'x-rapidapi-host': "yh-finance.p.rapidapi.com",
    'x-rapidapi-key': KEY
    }

response = requests.request("GET", balance_sheet_url, headers=headers, params=querystring)

json_response = response.json()

In [71]:
for x in json_response['balanceSheetHistoryQuarterly']['balanceSheetStatements']:
    print(x.keys())
    break

dict_keys(['intangibleAssets', 'capitalSurplus', 'totalLiab', 'totalStockholderEquity', 'otherCurrentLiab', 'totalAssets', 'endDate', 'otherCurrentAssets', 'retainedEarnings', 'otherLiab', 'goodWill', 'treasuryStock', 'otherAssets', 'cash', 'totalCurrentLiabilities', 'deferredLongTermAssetCharges', 'otherStockholderEquity', 'propertyPlantEquipment', 'totalCurrentAssets', 'netTangibleAssets', 'shortTermInvestments', 'netReceivables', 'maxAge', 'longTermDebt', 'accountsPayable'])


In [72]:
url = "https://yh-finance.p.rapidapi.com/stock/v2/get-financials"

querystring = {"symbol":"AMRN","region":"US"}

headers = {
    'x-rapidapi-host': "yh-finance.p.rapidapi.com",
    'x-rapidapi-key': KEY
    }

response = requests.request("GET", url, headers=headers, params=querystring).json()

In [78]:
##these are quarterly earnings specifically
revenue_df = pd.DataFrame()

for quarter in response['incomeStatementHistoryQuarterly']['incomeStatementHistory']:
    revenue_line = pd.Series([quarter['endDate']['fmt'], quarter['totalRevenue']['fmt']], index = ['quarter','revenue_billion'])
    revenue_df = revenue_df.append(revenue_line, ignore_index=True)

revenue_df

,quarter,revenue_billion
0,2021-09-30,142.04M
1,2021-06-30,154.49M
2,2021-03-31,142.17M
3,2020-12-31,167.25M


In [25]:
for x in json_response['balanceSheetHistoryQuarterly']['balanceSheetStatements']:
    print(x.keys())
    break

dict_keys(['intangibleAssets', 'capitalSurplus', 'totalLiab', 'totalStockholderEquity', 'otherCurrentLiab', 'totalAssets', 'endDate', 'otherCurrentAssets', 'retainedEarnings', 'otherLiab', 'goodWill', 'treasuryStock', 'otherAssets', 'cash', 'totalCurrentLiabilities', 'deferredLongTermAssetCharges', 'otherStockholderEquity', 'propertyPlantEquipment', 'totalCurrentAssets', 'netTangibleAssets', 'shortTermInvestments', 'netReceivables', 'maxAge', 'longTermDebt', 'accountsPayable'])


In [13]:
##these are quarterly earnings specifically
earnings_df = pd.DataFrame()

for quarter in json_response['earnings']['earningsChart']['quarterly']:
    earning_line = pd.Series([quarter['date'], quarter['actual']['raw']], index = ['quarter','raw_earnings_billion'])
    earnings_df = earnings_df.append(earning_line, ignore_index=True)

In [14]:
earnings_df

,quarter,raw_earning
0,4Q2020,1.21
1,1Q2021,0.84
2,2Q2021,0.91
3,3Q2021,0.72


In [53]:
##these are quarterly earnings specifically
balance_df = pd.DataFrame()

for item in json_response['balanceSheetHistoryQuarterly']['balanceSheetStatements']:
    balance_line = pd.Series([item['endDate']['fmt'], item['totalCurrentLiabilities']['fmt'], item['totalCurrentAssets']['fmt']], index = ['quarter','liabilities','assets'])
    balance_df = balance_df.append(balance_line, ignore_index = True)

In [59]:
balance_df.loc[:,'assets_billion'] = balance_df['assets'].str.strip('B').astype('float')
balance_df.loc[:,'liabilties_billion'] = balance_df['liabilities'].str.strip('B').astype('float')
balance_df.loc[:,'la_ratio'] = round(balance_df['liabilties_billion']/balance_df['assets_billion'],2)
balance_df = balance_df.drop(columns = ['assets','liabilities'])
balance_df

,quarter,assets_billion,liabilties_billion,la_ratio
0,2021-09-30,11.21,2.02,0.18
1,2021-06-30,10.89,2.25,0.21
2,2021-03-31,10.92,3.18,0.29
3,2020-12-31,10.56,3.10,0.29


In [25]:
def list_perfomance(symbol):
    weekly_history_df = get_weekly_history_df(symbol)

    performance_list = [symbol,\
                        round((weekly_history_df.iloc[0]['close'] - weekly_history_df.iloc[-1]['close'])/weekly_history_df.loc[0]['close'],3),\
                        round(weekly_history_df['close'].std(),2),\
                        round(weekly_history_df['close'].var(),2),
                        round(weekly_history_df['close'].mean(),2),
                        round(weekly_history_df['close'].std()/weekly_history_df['close'].mean(),2),
                        round(weekly_history_df.iloc[0]['close'],1)
                        ]

    return performance_list

In [26]:
counter = 0

attributes = ['ticker','yoy_growth','std','variance','mean_price','volatility','last_close_price']
comparison_df = pd.DataFrame(columns = attributes)

for ticker_name in nasdaq100_list[64:67]:
    series_row = pd.Series(list_perfomance(ticker_name), index = attributes)
    comparison_df = comparison_df.append(series_row, ignore_index=True)
    print(ticker_name, end = ' ')
    time.sleep(1)

In [27]:
comparison_df.sort_values(by = 'yoy_growth', ascending=False).head(20)

,ticker,yoy_growth,std,variance,mean_price,volatility,last_close_price
